In [1]:
import os
import sys
import json
import requests

from dotenv import load_dotenv

""" LangChain 관련 라이브러리 """
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, EnumOutputParser

""" Streamlit 관련 라이브러리 """
from enum import Enum

In [2]:
CATEGORIES = ["맛집", "관광지", "날씨", "검색"]

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
        
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

def llm_chain():
    
    response_schemas = [
        ResponseSchema(name="category", description="The selected category from the available categories", type="string")
    ]
    
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    
    format_instructions = output_parser.get_format_instructions()
    
    prompt = PromptTemplate.from_template(
        template = "입력한 문장을 분석하여, 다음의 카테고리 리스트에서 가장 가까운 카테고리 하나를 선택하시오.\n 카테고리 리스트: {categories}\n 출력 포맷:{format_instructions} \n\n 입력:{query}",
        partial_variables={"format_instructions": format_instructions, "categories": CATEGORIES},
    )
    

    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    chain = prompt | model | output_parser
    
    return chain

In [3]:
def categorize_menu(query):
    
    chain = llm_chain()
    
    return chain.invoke({"query": query})

In [5]:
# 0. API Key를 포함한 환경 변수 로드
setup_env()

# 1. Input User Query
user_query = input("Please input your query: ")

answered_category = categorize_menu(user_query)
# 2. Based on User Query, go to specific logic
print(type(answered_category))
print(answered_category)

Loaded environment variables from: /home/ws/src/kumdori_rag_exp/src/../.env
<class 'dict'>
{'category': '날씨'}
